In [ ]:
# Phase 1: Understanding the Error - Measured vs Theoretical Frequencies
# =========================================================================
# Goal: Compare your actual ukulele tuning against perfect equal temperament
# and understand where and why deviations occur.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set up plotting style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("🎸 Phase 1: Measured vs Theoretical - Let's see how your uke compares to math!")

In [ ]:
# Step 1: Load both datasets
# ---------------------------

# Get project root (two levels up from notebooks folder)
from python.shared.paths import get_project_root


project_root = get_project_root()
data_dir = project_root / "data"

# Load measured data (from your actual ukulele)
measured = pd.read_csv(data_dir / "measured_raw.csv")
measured = measured.rename(columns={'frequency_hz': 'measured_hz'})

# Load generated data (from mathematical theory)
generated = pd.read_csv(data_dir / "generated.csv")
generated = generated.rename(columns={'frequency_hz': 'theoretical_hz'})

# Merge them together
df = measured.merge(generated, on=['string', 'fret', 'note'], how='inner')

print(f"✅ Loaded {len(df)} measurements across {df['string'].nunique()} strings")
print(f"📊 Fret range: {df['fret'].min()} to {df['fret'].max()}")
print("\nFirst few rows:")
df.head(10)

In [ ]:
# Step 2: Calculate Cents Deviation
# -----------------------------------
# Cents are the REAL way to measure tuning error in music.
# 
# Why cents?
# - 1 octave = 1200 cents
# - 1 semitone = 100 cents
# - Human ear can detect ~5-10 cents deviation
# - Hz differences are misleading (20 Hz matters more at low frequencies)
#
# Formula: cents = 1200 * log2(measured / theoretical)

def hz_to_cents(measured_hz, theoretical_hz):
    """
    Calculate the deviation in cents between measured and theoretical frequency.
    
    Positive cents = sharp (too high)
    Negative cents = flat (too low)
    """
    return 1200 * np.log2(measured_hz / theoretical_hz)

# Calculate deviations
df['cents_deviation'] = hz_to_cents(df['measured_hz'], df['theoretical_hz'])
df['hz_deviation'] = df['measured_hz'] - df['theoretical_hz']

print("📐 Deviation Statistics:")
print(f"   Mean deviation: {df['cents_deviation'].mean():.2f} cents")
print(f"   Std deviation:  {df['cents_deviation'].std():.2f} cents")
print(f"   Max sharp:      {df['cents_deviation'].max():.2f} cents")
print(f"   Max flat:       {df['cents_deviation'].min():.2f} cents")
print(f"\n   (±5 cents = barely noticeable, ±20 cents = clearly off)")

df[['string', 'fret', 'note', 'measured_hz', 'theoretical_hz', 'cents_deviation']].head(13 * 4)

In [ ]:
# Step 3: Visualize the Error Pattern
# -------------------------------------
# Let's see WHERE on the fretboard your uke deviates from theory

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Cents deviation by fret and string
ax1 = axes[0]
for string in ['G', 'C', 'E', 'A']:
    string_data = df[df['string'] == string]
    ax1.plot(string_data['fret'], string_data['cents_deviation'], 
             marker='o', label=f'{string} string', linewidth=2, markersize=8)

ax1.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5, label='Perfect tuning')
ax1.axhline(y=5, color='green', linestyle=':', linewidth=1, alpha=0.3)
ax1.axhline(y=-5, color='green', linestyle=':', linewidth=1, alpha=0.3)
ax1.fill_between(range(13), -5, 5, alpha=0.1, color='green', label='±5 cents (acceptable)')

ax1.set_xlabel('Fret Number', fontsize=12, fontweight='bold')
ax1.set_ylabel('Cents Deviation', fontsize=12, fontweight='bold')
ax1.set_title('Tuning Deviation Across the Fretboard (Cents)', fontsize=14, fontweight='bold')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(range(13))

# Plot 2: Hz deviation by fret and string
ax2 = axes[1]
for string in ['G', 'C', 'E', 'A']:
    string_data = df[df['string'] == string]
    ax2.plot(string_data['fret'], string_data['hz_deviation'], 
             marker='s', label=f'{string} string', linewidth=2, markersize=8)

ax2.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax2.set_xlabel('Fret Number', fontsize=12, fontweight='bold')
ax2.set_ylabel('Hz Deviation', fontsize=12, fontweight='bold')
ax2.set_title('Frequency Deviation Across the Fretboard (Hz) - Notice how Hz "lies"', 
              fontsize=14, fontweight='bold')
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)
ax2.set_xticks(range(13))

plt.tight_layout()
plt.show()

print("\n💡 Key Insight:")
print("   Compare the two plots - same data, different units.")
print("   Hz deviation grows with frequency (higher frets = bigger Hz error)")
print("   But cents deviation reveals the MUSICAL truth - is it actually out of tune?")

In [ ]:
# Step 4: Heatmap - The Full Picture
# ------------------------------------
# Let's see the entire fretboard at once

# Pivot the data to create a fretboard-like view
heatmap_data = df.pivot(index='string', columns='fret', values='cents_deviation')

# Reorder strings to match physical layout (top to bottom on uke)
string_order = ['A', 'E', 'C', 'G']
heatmap_data = heatmap_data.reindex(string_order)

plt.figure(figsize=(16, 5))
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn_r', center=0,
            cbar_kws={'label': 'Cents Deviation'},
            linewidths=0.5, linecolor='gray')

plt.title('Fretboard Tuning Heatmap - Your Ukulele vs Perfect Theory', 
          fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Fret Number', fontsize=12, fontweight='bold')
plt.ylabel('String', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🎯 How to read this:")
print("   Green = close to perfect")
print("   Yellow/Orange = getting off")
print("   Red = significantly sharp or flat")
print("   Numbers show exact cents deviation")

In [ ]:
# Step 5: Distribution Analysis
# ------------------------------
# Let's understand the overall tuning quality

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of cents deviation
ax1 = axes[0]
ax1.hist(df['cents_deviation'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Perfect tuning')
ax1.axvline(x=df['cents_deviation'].mean(), color='orange', linestyle='--', 
            linewidth=2, label=f'Mean: {df["cents_deviation"].mean():.2f}¢')
ax1.set_xlabel('Cents Deviation', fontsize=12, fontweight='bold')
ax1.set_ylabel('Count', fontsize=12, fontweight='bold')
ax1.set_title('Distribution of Tuning Errors', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Box plot by string
ax2 = axes[1]
string_order = ['G', 'C', 'E', 'A']
df['string'] = pd.Categorical(df['string'], categories=string_order, ordered=True)
df_sorted = df.sort_values('string')

sns.boxplot(data=df_sorted, x='string', y='cents_deviation', ax=ax2, palette='Set2')
ax2.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax2.set_xlabel('String', fontsize=12, fontweight='bold')
ax2.set_ylabel('Cents Deviation', fontsize=12, fontweight='bold')
ax2.set_title('Tuning Consistency by String', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📊 Statistical Summary by String:")
print(df.groupby('string')['cents_deviation'].agg(['mean', 'std', 'min', 'max']).round(2))

In [ ]:
# Step 6: Find the Worst Offenders
# ----------------------------------
# Which notes are most out of tune?

print("🔴 Most Sharp (too high):")
print(df.nlargest(5, 'cents_deviation')[['string', 'fret', 'note', 'cents_deviation', 'hz_deviation']])

print("\n🔵 Most Flat (too low):")
print(df.nsmallest(5, 'cents_deviation')[['string', 'fret', 'note', 'cents_deviation', 'hz_deviation']])

print("\n✅ Closest to Perfect:")
df['abs_cents_deviation'] = df['cents_deviation'].abs()
print(df.nsmallest(5, 'abs_cents_deviation')[['string', 'fret', 'note', 'cents_deviation', 'hz_deviation']])

In [ ]:
# Step 7: Understanding WHY - Physical Factors
# ----------------------------------------------
# Let's explore what causes these deviations

print("🔬 Physical Factors That Cause Tuning Deviations:\n")

print("1. STRING STIFFNESS")
print("   - Real strings aren't perfectly flexible")
print("   - When you press down, you stretch the string slightly")
print("   - This makes notes sharper than theory predicts")
print("   - Effect increases with fret number (more stretch)\n")

print("2. FRET PLACEMENT")
print("   - Frets must be placed with millimeter precision")
print("   - Manufacturing tolerances can cause small errors")
print("   - Cumulative effect grows up the neck\n")

print("3. NUT & SADDLE COMPENSATION")
print("   - The nut (top) and saddle (bottom) affect string length")
print("   - Proper 'intonation' requires compensation")
print("   - Your uke might need adjustment here\n")

print("4. FINGER PRESSURE")
print("   - How hard you press affects pitch")
print("   - You might have pressed harder on some measurements")
print("   - This is why cents analysis is more reliable than Hz\n")

print("5. STRING AGE & QUALITY")
print("   - Old strings lose elasticity")
print("   - Cheap strings have inconsistent density")
print("   - Both affect tuning stability\n")

# Let's check if there's a pattern with fret number
correlation = df['fret'].corr(df['cents_deviation'])
print(f"📈 Correlation between fret number and deviation: {correlation:.3f}")
if abs(correlation) > 0.5:
    print("   → Strong correlation! Higher frets tend to be more off.")
elif abs(correlation) > 0.3:
    print("   → Moderate correlation. Some pattern exists.")
else:
    print("   → Weak correlation. Deviations are fairly random.")

In [ ]:
# Step 8: Phase 1 Summary & Key Insights
# ----------------------------------------

print("=" * 70)
print("🎸 PHASE 1 COMPLETE: Measured vs Theoretical Analysis")
print("=" * 70)

# Calculate overall tuning quality
within_5_cents = (df['abs_cents_deviation'] <= 5).sum()
within_10_cents = (df['abs_cents_deviation'] <= 10).sum()
within_20_cents = (df['abs_cents_deviation'] <= 20).sum()
total = len(df)

print(f"\n📊 Overall Tuning Quality:")
print(f"   Within ±5 cents (excellent):  {within_5_cents}/{total} ({within_5_cents/total*100:.1f}%)")
print(f"   Within ±10 cents (good):      {within_10_cents}/{total} ({within_10_cents/total*100:.1f}%)")
print(f"   Within ±20 cents (acceptable): {within_20_cents}/{total} ({within_20_cents/total*100:.1f}%)")

print(f"\n🎯 Key Findings:")
print(f"   • Average deviation: {df['cents_deviation'].mean():.2f} cents")
print(f"   • Standard deviation: {df['cents_deviation'].std():.2f} cents")
print(f"   • Largest error: {df['abs_cents_deviation'].max():.2f} cents")

# Determine overall verdict
avg_abs_deviation = df['abs_cents_deviation'].mean()
if avg_abs_deviation < 5:
    verdict = "EXCELLENT - Your uke is very well tuned! 🎉"
elif avg_abs_deviation < 10:
    verdict = "GOOD - Your uke is well within acceptable range 👍"
elif avg_abs_deviation < 20:
    verdict = "ACCEPTABLE - Playable but could use some adjustment 🔧"
else:
    verdict = "NEEDS WORK - Consider professional setup 🛠️"

print(f"\n🏆 Verdict: {verdict}")

print("\n💡 What We Learned:")
print("   1. Cents (not Hz) are the true measure of musical tuning")
print("   2. Your measurements show the gap between theory and reality")
print("   3. Small deviations are normal and expected")
print("   4. The pattern of errors reveals physical properties of your instrument")

print("\n➡️  Ready for Phase 2: Exploring the geometric structure of music!")
print("=" * 70)

In [ ]:
# =========================================================================
# PHASE 2: THE GEOMETRY OF MUSIC
# =========================================================================
# Goal: Reveal the hidden structure - why music "works" mathematically
# 
# We'll discover:
# 1. The log-linear nature of pitch (the "ahhh" moment)
# 2. Pitch class vs octave separation (the circular + linear structure)
# 3. Note aliases across the fretboard (same pitch, different fingers)
# 4. Intervals and their mathematical relationships
# 5. Where chords emerge from geometry

print("\n" + "="*70)
print("🎼 PHASE 2: EXPLORING THE GEOMETRIC STRUCTURE OF MUSIC")
print("="*70)

In [ ]:
# Step 1: The "Ahhh" Moment - Log-Linear Pitch
# ----------------------------------------------
# Frequency grows exponentially, but PITCH grows linearly
# This is why cents work, and why music "makes sense"

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Frequency vs Fret (exponential)
ax1 = axes[0]
for string in ['G', 'C', 'E', 'A']:
    string_data = df[df['string'] == string].sort_values('fret')
    ax1.plot(string_data['fret'], string_data['theoretical_hz'], 
             marker='o', label=f'{string} string', linewidth=2, markersize=6)

ax1.set_xlabel('Fret Number', fontsize=12, fontweight='bold')
ax1.set_ylabel('Frequency (Hz)', fontsize=12, fontweight='bold')
ax1.set_title('Frequency vs Fret - Exponential Growth', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xticks(range(13))

# Plot 2: Log2(Frequency) vs Fret (LINEAR!)
ax2 = axes[1]
for string in ['G', 'C', 'E', 'A']:
    string_data = df[df['string'] == string].sort_values('fret')
    log_freq = np.log2(string_data['theoretical_hz'])
    ax2.plot(string_data['fret'], log_freq, 
             marker='o', label=f'{string} string', linewidth=2, markersize=6)

ax2.set_xlabel('Fret Number', fontsize=12, fontweight='bold')
ax2.set_ylabel('Log₂(Frequency)', fontsize=12, fontweight='bold')
ax2.set_title('Log₂(Frequency) vs Fret - LINEAR! 🎯', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xticks(range(13))

plt.tight_layout()
plt.show()

print("\n💡 THE 'AHHH' MOMENT:")
print("   Left:  Frequency grows exponentially (curves up)")
print("   Right: Log₂(frequency) grows linearly (straight lines!)")
print("\n   This is WHY:")
print("   • Each fret is the same MUSICAL distance (1 semitone)")
print("   • But different FREQUENCY distance (depends on starting point)")
print("   • Log space reveals the TRUE structure")
print("   • This is how your brain actually hears pitch!")
print("\n   The lines are parallel because each string follows the SAME rule:")
print("   f(n) = f₀ × 2^(n/12)")

In [ ]:
# Step 2: Separating Pitch Class from Octave
# --------------------------------------------
# Notes are CIRCULAR (A, A#, B, C... wraps back to A)
# Octaves are LINEAR (each octave is 2x frequency)
# Let's make this explicit

# Define the chromatic sequence
chromatic = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']

def get_pitch_class(note):
    """Extract just the note name (without octave)"""
    return note

def get_octave(frequency):
    """Calculate which octave this frequency belongs to"""
    # A4 = 440 Hz is our reference (octave 4)
    # Each octave doubles frequency
    octave = 4 + np.log2(frequency / 440)
    return round(octave, 2)

def get_pitch_class_index(note):
    """Get the position in chromatic scale (0-11)"""
    return chromatic.index(note)

# Add these columns to our dataframe
df['pitch_class'] = df['note'].apply(get_pitch_class)
df['octave'] = df['theoretical_hz'].apply(get_octave)
df['pitch_class_index'] = df['pitch_class'].apply(get_pitch_class_index)

print("🎵 Enriched Dataset with Pitch Class + Octave:")
print(df[['string', 'fret', 'note', 'theoretical_hz', 'pitch_class', 'octave', 'pitch_class_index']].head(13 * 4))

print("\n📊 Unique pitch classes:", sorted(df['pitch_class'].unique()))
print(f"📊 Octave range: {df['octave'].min():.1f} to {df['octave'].max():.1f}")
print(f"📊 Total unique notes: {len(df)}")
print(f"📊 Unique pitch classes: {df['pitch_class'].nunique()}")

In [ ]:
# Step 3: Finding Note Aliases - Same Pitch, Different Positions
# ----------------------------------------------------------------
# The fretboard has REDUNDANCY - you can play the same note in multiple places
# This is crucial for chord voicings and playing technique

# Group by pitch class and find all positions
print("🔍 NOTE ALIASES - Where the same pitch appears on different strings:\n")

# Let's find notes that appear on multiple strings
pitch_positions = df.groupby('pitch_class').apply(
    lambda x: x[['string', 'fret', 'theoretical_hz']].to_dict('records')
).to_dict()

# Show some interesting examples
interesting_notes = ['C', 'D', 'E', 'F', 'G','A', 'B']

for note in interesting_notes:
    positions = pitch_positions[note]
    print(f"\n{note} can be played at:")
    for pos in positions:
        print(f"   • {pos['string']} string, fret {pos['fret']} ({pos['theoretical_hz']:.2f} Hz)")

print("\n💡 Why This Matters:")
print("   • Same note, different timbres (string thickness affects tone)")
print("   • Enables chord voicings (choose which string to use)")
print("   • Allows position shifts while maintaining melody")
print("   • This redundancy is a FEATURE, not a bug!")

In [ ]:
# Step 4: Visualizing the Fretboard as Pitch Space
# --------------------------------------------------
# Let's see the entire fretboard colored by pitch class
# This reveals the PATTERN

# Create a pivot for visualization
fretboard_pitch = df.pivot(index='string', columns='fret', values='pitch_class_index')
string_order = ['A', 'E', 'C', 'G']
fretboard_pitch = fretboard_pitch.reindex(string_order)

# Create custom colormap for the 12 pitch classes
fig, ax = plt.subplots(figsize=(16, 6))

# Plot with pitch class colors
im = ax.imshow(fretboard_pitch, cmap='hsv', aspect='auto', interpolation='none')

# Add note names as text
for i, string in enumerate(string_order):
    for fret in range(13):
        note = df[(df['string'] == string) & (df['fret'] == fret)]['note'].values[0]
        pitch_idx = df[(df['string'] == string) & (df['fret'] == fret)]['pitch_class_index'].values[0]
        
        # Choose text color based on background
        text_color = 'white' if pitch_idx in [0, 1, 5, 6, 7] else 'black'
        
        ax.text(fret, i, note, ha='center', va='center', 
                fontsize=11, fontweight='bold', color=text_color)

ax.set_xticks(range(13))
ax.set_yticks(range(4))
ax.set_xticklabels(range(13))
ax.set_yticklabels(string_order)
ax.set_xlabel('Fret Number', fontsize=12, fontweight='bold')
ax.set_ylabel('String', fontsize=12, fontweight='bold')
ax.set_title('Fretboard as Pitch Space - Same Colors = Same Pitch Class', 
             fontsize=14, fontweight='bold', pad=20)

# Add colorbar
cbar = plt.colorbar(im, ax=ax, ticks=range(12))
cbar.set_label('Pitch Class', fontsize=11, fontweight='bold')
cbar.ax.set_yticklabels(chromatic)

plt.tight_layout()
plt.show()

print("\n🎨 Pattern Recognition:")
print("   • Same colors = same pitch class (different octaves)")
print("   • Diagonal patterns = intervals")
print("   • The pattern REPEATS every 12 frets (one octave)")
print("   • This is the GEOMETRY of music!")

# Melody: G - B - D - E - D - D - D - C - E - F - G - F - G - A - G - C

In [ ]:
# Step 5: Understanding Intervals - The Distances Between Notes
# ---------------------------------------------------------------
# An interval is the musical distance between two notes
# Let's calculate and understand them

def calculate_interval_semitones(note1_idx, note2_idx):
    """Calculate interval in semitones (always positive)"""
    return abs(note2_idx - note1_idx)

def calculate_interval_cents(freq1, freq2):
    """Calculate interval in cents"""
    result = abs(1200 * np.log2(freq2 / freq1))
    return round(result, 2)

def calculate_frequency_ratio(freq1, freq2):
    """Calculate the frequency ratio"""
    ratio = max(freq1, freq2) / min(freq1, freq2)
    return ratio

# Let's analyze intervals from the open C string (middle reference)
def draw_intervals(base_string: str):

    string_notes = df[df['string'] == base_string].sort_values('fret')
    base_freq = string_notes[string_notes['fret'] == 0]['theoretical_hz'].values[0]
    base_note = string_notes[string_notes['fret'] == 0]['note'].values[0]

    intervals_data = []
    for _, row in string_notes.iterrows():
        semitones = row['fret']
        freq = row['theoretical_hz']
        ratio = calculate_frequency_ratio(base_freq, freq)
        cents = calculate_interval_cents(base_freq, freq)
        
        intervals_data.append({
            'fret': row['fret'],
            'note': row['note'],
            'semitones': semitones,
            'frequency_ratio': ratio,
            'cents': cents,
            'frequency_hz': freq
        })

    intervals_df = pd.DataFrame(intervals_data)

    print(f"🎼 INTERVALS FROM {base_note} ({base_string} string, open):\n")
    print(intervals_df.to_string(index=False))

    print("\n\n📚 Musical Interval Names:")
    interval_names = {
        0: 'Unison (same note)',
        1: 'Minor 2nd (semitone)',
        2: 'Major 2nd (whole tone)',
        3: 'Minor 3rd',
        4: 'Major 3rd',
        5: 'Perfect 4th',
        6: 'Tritone (augmented 4th)',
        7: 'Perfect 5th',
        8: 'Minor 6th',
        9: 'Major 6th',
        10: 'Minor 7th',
        11: 'Major 7th',
        12: 'Octave (2:1 ratio)'
    }

    for semitones, name in interval_names.items():
        if semitones < len(intervals_df):
            ratio = intervals_df.iloc[semitones]['frequency_ratio']
            print(f"   {semitones:2d} semitones: {name:25s} (ratio: {ratio:.4f}:1)\n")
    
    return intervals_df

intervals_map = {}

for string in ['A', 'E', 'C', 'G']:
    intervals_map[string] = draw_intervals(string)

In [ ]:
# Step 6: Harmonic Relationships - Why Some Intervals "Sound Good"
# ------------------------------------------------------------------
# Consonant intervals have SIMPLE frequency ratios
# Dissonant intervals have COMPLEX frequency ratios
# Let's discover this pattern

print("🎵 HARMONIC ANALYSIS - The Math Behind 'Consonance'\n")
print("="*70)

# Perfect intervals (most consonant)
perfect_intervals = {
    'Octave': (12, 2.0),           # 2:1 ratio
    'Perfect 5th': (7, 1.5),       # 3:2 ratio (approximately)
    'Perfect 4th': (5, 4/3),       # 4:3 ratio (approximately)
}

# Major/Minor intervals
major_minor = {
    'Major 3rd': (4, 5/4),         # 5:4 ratio (approximately)
    'Minor 3rd': (3, 6/5),         # 6:5 ratio (approximately)
    'Major 6th': (9, 5/3),         # 5:3 ratio (approximately)
    'Minor 6th': (8, 8/5),         # 8:5 ratio (approximately)
}

# Dissonant intervals
dissonant = {
    'Tritone': (6, None),          # Most dissonant
    'Minor 2nd': (1, None),        # Very dissonant
    'Major 7th': (11, None),       # Dissonant
}


def draw_consonance_scores(intervals_df: pd.DataFrame):
    print("\n🟢 CONSONANT INTERVALS (Simple Ratios):")
    print("-" * 70)
    for name, (semitones, ideal_ratio) in perfect_intervals.items():
        actual_ratio = intervals_df.iloc[semitones]['frequency_ratio']
        if ideal_ratio:
            error = abs(actual_ratio - ideal_ratio) / ideal_ratio * 100
            print(f"{name:20s} ({semitones:2d} semitones)")
            print(f"   Ideal ratio:  {ideal_ratio:.6f}:1")
            print(f"   Actual ratio: {actual_ratio:.6f}:1")
            print(f"   Error:        {error:.4f}%")
            print()

    print("\n🟡 MAJOR/MINOR INTERVALS (Moderately Simple Ratios):")
    print("-" * 70)
    for name, (semitones, ideal_ratio) in major_minor.items():
        actual_ratio = intervals_df.iloc[semitones]['frequency_ratio']
        if ideal_ratio:
            error = abs(actual_ratio - ideal_ratio) / ideal_ratio * 100
            print(f"{name:20s} ({semitones:2d} semitones)")
            print(f"   Ideal ratio:  {ideal_ratio:.6f}:1")
            print(f"   Actual ratio: {actual_ratio:.6f}:1")
            print(f"   Error:        {error:.4f}%")
            print()

    print("\n🔴 DISSONANT INTERVALS (Complex Ratios):")
    print("-" * 70)
    for name, (semitones, _) in dissonant.items():
        actual_ratio = intervals_df.iloc[semitones]['frequency_ratio']
        print(f"{name:20s} ({semitones:2d} semitones)")
        print(f"   Ratio: {actual_ratio:.6f}:1 (no simple integer ratio)")
        print()

for string in ['C']:
    draw_consonance_scores(intervals_map[string])

print("\n💡 THE PATTERN:")
print("   • Consonant intervals ≈ simple ratios (2:1, 3:2, 4:3, 5:4)")
print("   • Dissonant intervals ≈ complex ratios (no small integers)")
print("   • Your brain recognizes simple patterns as 'pleasing'")
print("   • Complex patterns sound 'tense' or 'unstable'")
print("\n   This is why music theory works - it's based on math your brain likes!")

In [ ]:
# Step 7: Visualizing Interval Consonance
# -----------------------------------------
# Let's plot how "simple" each interval's ratio is

# Calculate how close each interval is to simple ratios
simple_ratios = {
    0: (1, 1),    # Unison
    5: (4, 3),    # Perfect 4th
    7: (3, 2),    # Perfect 5th
    12: (2, 1),   # Octave
    4: (5, 4),    # Major 3rd
    3: (6, 5),    # Minor 3rd
    9: (5, 3),    # Major 6th
    8: (8, 5),    # Minor 6th
}

def draw_fretboard_with_consonance_scores(intervals_df: pd.DataFrame, base_note: str):
    consonance_scores = []
    for _, row in intervals_df.iterrows():
        semitones = row['semitones']
        actual_ratio = row['frequency_ratio']
        
        if semitones in simple_ratios:
            num, den = simple_ratios[semitones]
            ideal_ratio = num / den
            # Consonance = how close to simple ratio (inverse of error)
            error = abs(actual_ratio - ideal_ratio) / ideal_ratio
            consonance = 1 / (1 + error * 100)  # Higher = more consonant
        else:
            consonance = 0.1  # Dissonant intervals get low score
        
        consonance_scores.append(consonance)

    intervals_df['consonance'] = consonance_scores

    # Plot
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))

    # Plot 1: Frequency ratios
    ax1 = axes[0]
    bars1 = ax1.bar(intervals_df['semitones'], intervals_df['frequency_ratio'], 
                    color='steelblue', edgecolor='black', linewidth=1.5)
    ax1.set_xlabel('Interval (Semitones)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Frequency Ratio', fontsize=12, fontweight='bold')
    ax1.set_title(f'Frequency Ratios of Musical Intervals {base_note}', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='y')
    ax1.set_xticks(range(13))

    # Highlight perfect intervals
    for semitones in [0, 5, 7, 12]:
        if semitones < len(bars1):
            bars1[semitones].set_color('darkgreen')
            bars1[semitones].set_alpha(0.8)

    # Plot 2: Consonance scores
    ax2 = axes[1]
    colors = ['darkgreen' if s in [0, 5, 7, 12] else 
            'orange' if s in [3, 4, 8, 9] else 
            'red' for s in intervals_df['semitones']]

    bars2 = ax2.bar(intervals_df['semitones'], intervals_df['consonance'], 
                    color=colors, edgecolor='black', linewidth=1.5, alpha=0.7)
    ax2.set_xlabel('Interval (Semitones)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Consonance Score', fontsize=12, fontweight='bold')
    ax2.set_title('Interval Consonance - Green=Consonant, Orange=Moderate, Red=Dissonant', 
                fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    ax2.set_xticks(range(13))
    ax2.set_ylim(0, 1.1)

    # Add interval names
    interval_labels = ['U', 'm2', 'M2', 'm3', 'M3', 'P4', 'TT', 'P5', 'm6', 'M6', 'm7', 'M7', 'Oct']
    ax2.set_xticklabels(interval_labels, rotation=0)

    plt.tight_layout()
    plt.show()

for string in ['C']:
    draw_fretboard_with_consonance_scores(intervals_map[string], string)

print("\n🎼 Interval Legend:")
print("   U=Unison, m2=Minor 2nd, M2=Major 2nd, m3=Minor 3rd, M3=Major 3rd")
print("   P4=Perfect 4th, TT=Tritone, P5=Perfect 5th, m6=Minor 6th, M6=Major 6th")
print("   m7=Minor 7th, M7=Major 7th, Oct=Octave")

In [ ]:
# Step 8: Chord Discovery - Geometry of Common Chords
# -----------------------------------------------------
# Chords are just SETS of intervals played together
# Let's see how common chords emerge from the math

print("🎸 CHORD GEOMETRY - How Chords Emerge from Intervals\n")
print("="*70)

# Define common chord types by their interval structure (in semitones from root)
chord_types = {
    'Major': [0, 4, 7],           # Root, Major 3rd, Perfect 5th
    'Minor': [0, 3, 7],           # Root, Minor 3rd, Perfect 5th
    'Diminished': [0, 3, 6],      # Root, Minor 3rd, Tritone
    'Augmented': [0, 4, 8],       # Root, Major 3rd, Minor 6th
    'Major 7th': [0, 4, 7, 11],   # Major + Major 7th
    'Minor 7th': [0, 3, 7, 10],   # Minor + Minor 7th
    'Dominant 7th': [0, 4, 7, 10],# Major + Minor 7th
}

# Let's analyze a C major chord as example
print("\n🎵 C MAJOR CHORD ANALYSIS:")
print("-" * 70)
c_major_intervals = chord_types['Major']
c_freq = 261.63  # C note

print("Notes in C Major: C - E - G")
print("\nInterval Structure:")
for i, semitones in enumerate(c_major_intervals):
    freq = c_freq * (2 ** (semitones / 12))
    note_idx = (chromatic.index('C') + semitones) % 12
    note = chromatic[note_idx]
    
    if i == 0:
        interval_name = "Root"
    elif semitones == 4:
        interval_name = "Major 3rd"
    elif semitones == 7:
        interval_name = "Perfect 5th"
    else:
        interval_name = f"{semitones} semitones"
    
    print(f"   {note:3s} ({freq:6.2f} Hz) - {interval_name}")

# Calculate ratios between chord tones
print("\nFrequency Ratios within the chord:")
freqs = [c_freq * (2 ** (s / 12)) for s in c_major_intervals]
for i in range(len(freqs)):
    for j in range(i+1, len(freqs)):
        ratio = freqs[j] / freqs[i]
        note1_idx = (chromatic.index('C') + c_major_intervals[i]) % 12
        note2_idx = (chromatic.index('C') + c_major_intervals[j]) % 12
        note1 = chromatic[note1_idx]
        note2 = chromatic[note2_idx]
        print(f"   {note1} to {note2}: {ratio:.4f}:1")

print("\n💡 Why C Major Sounds 'Happy' and 'Stable':")
print("   • Contains Perfect 5th (3:2 ratio) - very consonant")
print("   • Contains Major 3rd (5:4 ratio) - bright, consonant")
print("   • All intervals are consonant (no dissonance)")
print("   • Simple ratios = brain recognizes pattern easily")

# Compare with C minor
print("\n\n🎵 C MINOR CHORD ANALYSIS:")
print("-" * 70)
c_minor_intervals = chord_types['Minor']

print("Notes in C Minor: C - Eb - G")
print("\nInterval Structure:")
for i, semitones in enumerate(c_minor_intervals):
    freq = c_freq * (2 ** (semitones / 12))
    note_idx = (chromatic.index('C') + semitones) % 12
    note = chromatic[note_idx]
    
    if i == 0:
        interval_name = "Root"
    elif semitones == 3:
        interval_name = "Minor 3rd"
    elif semitones == 7:
        interval_name = "Perfect 5th"
    else:
        interval_name = f"{semitones} semitones"
    
    print(f"   {note:3s} ({freq:6.2f} Hz) - {interval_name}")

print("\n💡 Why C Minor Sounds 'Sad' or 'Dark':")
print("   • Still has Perfect 5th (stable)")
print("   • But Minor 3rd (6:5 ratio) instead of Major 3rd")
print("   • Slightly more complex ratio = different emotional quality")
print("   • Same structure, one note changed = completely different feel!")

In [ ]:
# Step 9: Finding Chord Shapes on Your Ukulele
# ----------------------------------------------
# Now let's find WHERE you can play these chords on the fretboard

def find_chord_voicings(root_note, chord_intervals, max_fret_span=4):
    """
    Find possible ways to play a chord on the ukulele
    
    Parameters:
    - root_note: the root note of the chord (e.g., 'C')
    - chord_intervals: list of semitones from root (e.g., [0, 4, 7] for major)
    - max_fret_span: maximum stretch between lowest and highest fret
    
    Returns list of possible voicings
    """
    # Calculate which notes we need
    root_idx = chromatic.index(root_note)
    chord_notes = [(root_idx + interval) % 12 for interval in chord_intervals]
    chord_note_names = [chromatic[idx] for idx in chord_notes]
    
    voicings = []
    
    # For each possible position on the fretboard
    for base_fret in range(0, 10):  # Don't go too high
        # Try to find a voicing starting around this fret
        voicing = {'frets': [], 'notes': [], 'base_fret': base_fret}
        
        for string in ['G', 'C', 'E', 'A']:
            # Find the closest chord tone on this string near base_fret
            string_data = df[df['string'] == string]
            
            # Look for chord tones within reasonable fret range
            for fret in range(max(0, base_fret - 2), min(13, base_fret + max_fret_span + 2)):
                note_data = string_data[string_data['fret'] == fret]
                if len(note_data) > 0:
                    note = note_data['note'].values[0]
                    if note in chord_note_names:
                        voicing['frets'].append((string, fret))
                        voicing['notes'].append(note)
                        break
        
        # Check if we got all strings and it's a valid voicing
        if len(voicing['frets']) == 4:
            fret_numbers = [f[1] for f in voicing['frets']]
            fret_span = max(fret_numbers) - min([f for f in fret_numbers if f > 0] or [0])
            
            if fret_span <= max_fret_span:
                voicings.append(voicing)
    
    # Remove duplicates
    unique_voicings = []
    seen = set()
    for v in voicings:
        fret_tuple = tuple(f[1] for f in v['frets'])
        if fret_tuple not in seen:
            seen.add(fret_tuple)
            unique_voicings.append(v)
    
    return unique_voicings[:5]  # Return first 5 unique voicings

# Find major chord voicings
for base_note in [
    #'C',
    #'D',
    #'E',
    'F', 
    #'G',
    #'A',
    #'B'
    ]:
    print(f"🎸 {base_note} MAJOR CHORD VOICINGS ON UKULELE:\n")
    major_voicings = find_chord_voicings(base_note, [0, 4, 7])

    for i, voicing in enumerate(major_voicings, 1):
        print(f"Voicing {i}:")
        print("   String | Fret | Note")
        print("   -------|------|-----")
        for (string, fret), note in zip(voicing['frets'], voicing['notes']):
            print(f"   {string:6s} | {fret:4d} | {note}")
        print()


print("\n💡 How to Read This:")
print("   • Each voicing shows which fret to press on each string")
print("   • Fret 0 = open string (don't press)")
print("   • These are actual playable chord shapes!")
print("   • Different voicings = same chord, different positions")

In [ ]:
# Step 10: Phase 2 Summary - The Geometry Revealed
# --------------------------------------------------

print("\n" + "="*70)
print("🎼 PHASE 2 COMPLETE: THE GEOMETRY OF MUSIC")
print("="*70)

print("\n🎯 What We Discovered:\n")

print("1. THE LOG-LINEAR STRUCTURE")
print("   • Frequency grows exponentially (f = f₀ × 2^(n/12))")
print("   • But PITCH grows linearly (log₂ space reveals straight lines)")
print("   • This is why cents work and why music 'makes sense'")
print("   • Your brain hears in log space!")

print("\n2. PITCH CLASS vs OCTAVE")
print("   • Notes are CIRCULAR (12 pitch classes, then repeat)")
print("   • Octaves are LINEAR (each octave = 2x frequency)")
print("   • Music has both circular and linear structure")
print("   • This duality creates the fretboard pattern")

print("\n3. NOTE ALIASES & REDUNDANCY")
print("   • Same note can be played in multiple positions")
print("   • This is a FEATURE - enables voicings and technique")
print("   • Different strings = same pitch, different timbre")

print("\n4. INTERVALS ARE RATIOS")
print("   • Musical intervals = frequency ratios")
print("   • Consonant intervals ≈ simple ratios (2:1, 3:2, 5:4)")
print("   • Dissonant intervals ≈ complex ratios")
print("   • Your brain recognizes simple patterns as 'pleasing'")

print("\n5. CHORDS ARE INTERVAL SETS")
print("   • Major chord = [0, 4, 7] semitones (root, M3, P5)")
print("   • Minor chord = [0, 3, 7] semitones (root, m3, P5)")
print("   • One note difference = completely different emotion")
print("   • Chords are just geometry on the fretboard")

print("\n6. THE PATTERN REPEATS")
print("   • Every 12 frets = one octave")
print("   • Same pattern, higher pitch")
print("   • The fretboard is a VISUAL MAP of pitch space")

print("\n\n💡 The Big Insight:")
print("   Music theory isn't arbitrary rules - it's EMERGENT from simple math.")
print("   ")
print("   • Start with: f(n) = f₀ × 2^(n/12)")
print("   • Get: all of Western music theory")
print("   • Intervals, chords, consonance, dissonance - all from ONE formula")
print("   ")
print("   Simple rules → complex patterns → music")

print("\n🎸 Practical Takeaways for Playing Ukulele:")
print("   • Understanding intervals helps you find notes quickly")
print("   • Knowing chord geometry helps you create voicings")
print("   • Seeing the pattern helps you transpose and improvise")
print("   • Math and music are the same thing, just different languages")

print("\n➡️  Next: Phase 3 - Practical Applications & Exploration")
print("="*70)